## <center> Lista 10 - Aprendizado de Máquina </center>

**Aluno(a):** Marianna de Pinho Severo <br>
**Matrícula:** 374856 <br>
**Professor(a):** Regis Pires

Suponha que a internet caiu no Campus Quixadá e você quer fazer uma consulta através de palavras chave sobre as reviews de filmes do IMDB. O objetivo dessa lista é aplicarmos os conceitos de distância e processamendo de texto, mostrando as 10 reviews mais semelhantes a uma consulta que realizaremos, a qual será composta por algumas palavras-chave.

### Passo 01: Importar Bibliotecas

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import re
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
#nltk.download()

## 1) Definir consulta

Aqui definiremos as palavras-chave que servirão de comparação com as reviews.

In [2]:
consultas = 'good soundtrack great interesting scenes lovely need improvements movie'
consultas

'good soundtrack great interesting scenes lovely need improvements movie'

### Passo 02: Carregar conjunto de dados

In [3]:
reviews = pd.read_csv("movie_data.csv.gz")

In [4]:
reviews.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


### Passo 03: Capturar apenas uma parte do conjunto de dados para ser usada

Conforme podemos observar abaixo, o conjunto de dados possui 50000 documentos, ou seja, linhas. Isso faz com que o poder computacional necessário para processar todos esses dados seja um pouco mais elevado, aumentando o tempo para a realização das atividades. Dessa forma, utilizaremos apenas uma parte do conjunto de dados, com o objetivo de ganhar tempo enquanto colocamos em prática os conhecimentos cobrados nesta lista.

In [5]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null int64
dtypes: int64(1), object(1)
memory usage: 781.3+ KB


In [6]:
reviews = reviews[:1000]

Além das 1000 linhas do conjunto de dados, adicionaremos mais uma linha, correspondente à consulta que desejamos realizar.

In [7]:
reviews.loc[1000] = consultas

In [8]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 0 to 1000
Data columns (total 2 columns):
review       1001 non-null object
sentiment    1001 non-null object
dtypes: object(2)
memory usage: 23.5+ KB


### Passo 04: Limpar o conjunto de dados

Conforme podemos ver no exemplo abaixo, o conjunto de dados vem com algumas expressões que não são desejadas, como, por exemplo, tags html. Então, para que possamos aproveitar melhor os textos que possuímos, precisamos limpá-los, retirando todas as expressões que não são desejadas.

In [9]:
piece = reviews.loc[0, 'review'][-50:]
piece

'is seven.<br /><br />Title (Brazil): Not Available'

Para limparmos os textos, utilizaremos a função abaixo, cuja implementação foi retirada de [Chapter 8 - Applying Machine Learning To Sentiment Analysis](https://nbviewer.jupyter.org/github/rasbt/python-machine-learning-book-2nd-edition/blob/master/code/ch08/ch08.ipynb).

In [10]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()))
    return text

Aplicando a função **preprocessor** no pedaço de texto que separamos acima, obtemos o seguinte resultado:

In [11]:
preprocessor(piece)

'is seven title brazil not available'

Agora, aplicaremos o **preprocessor** em todo o conjunto de dados.

In [12]:
reviews['review'] = reviews['review'].apply(preprocessor)

In [13]:
reviews.loc[0, 'review'][-50:]

'zation my vote is seven title brazil not available'

### Passo 05: Stemização dos dados

Em um texto, várias palavras são derivadas uma das outras, apresentando aproximadamente o mesmo sentido. Isso é melhor explicado em [Stemming and Lemmatization in Python](https://www.datacamp.com/community/tutorials/stemming-lemmatization-python). Então, realizaremos a Stemização, ou normalização do texto, contribuindo para o cálculo de sua similaridade. Para isso, utilizaremos a função abaixo, retirada do mesmo link já citado.

In [14]:
porter = PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [15]:
reviews.head()

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


In [16]:
text_review = reviews['review'].values
normalized_text = []
for text in text_review:
    normalized_text.append(stemSentence(text))

In [17]:
 reviews['review'] = normalized_text

In [18]:
reviews.head()

,review,sentiment
0,in 1974 the teenag martha moxley maggi grace m...,1
1,ok so i realli like kri kristofferson and hi u...,0
2,spoiler do not read thi if you think about wat...,0
3,hi for all the peopl who have seen thi wonder ...,1
4,i recent bought the dvd forget just how much i...,0


### Passo 06: Vetorizar reviews

Uma das formas que temos para trabalhar com processamento de texto em aprendizado de máquina é através de sua tokenização. Isso significa que, dado um *corpus*, que é um conjunto de dados de textos, para cada *documento*, que é cada linha desse conjunto de dados, separamos cada palavra de cada documento, criando *tokens*.

Utilizando a classe  **TfidfVectorizer** podemos, além de realizar a tokenização de documentos, dizer que tokens dentro de um conjunto de dados tem maior relevância, aproveitando essa informação para diferentes tipos de aplicações. Com ela, para cada linha do conjunto de dados, atribuimos um valor a cada palavra contida no documento. Esse valor é o produto da frequência do termo (palavra) dentro do documento e o inverso da frequência desse mesmo termo entre os documentos. Ou seja, levamos em consideração tanto a quantidade de vezes que uma palavra aparece dentro de um documento, como também a quantidade de vezes que ela aparece nos outros documentos. Quanto mais essa palavra estiver nos outros documentos, menor será o valor atribuído. Ao passo que, quanto mais essa palavra aparecer no documento em observação (documento atual), maior será o valor atribuído.

Ao final, teremos uma matriz em que as linhas representam os documentos, as colunas representam os tokens e os valores são aqueles resultantes da aplicação da TfidfVectorizer.

In [19]:
reviews.head()

,review,sentiment
0,in 1974 the teenag martha moxley maggi grace m...,1
1,ok so i realli like kri kristofferson and hi u...,0
2,spoiler do not read thi if you think about wat...,0
3,hi for all the peopl who have seen thi wonder ...,1
4,i recent bought the dvd forget just how much i...,0


In [20]:
documents = reviews['review'].values
documents[-2:]

array(['thi begin a wager between edgar allen poe and a journalist poe bet that the man can not spend an entir night in a creepi castl well of cours he can but will he come out unscath hard to say with all these strang peopl that aren t suppos to be there wander around includ the ici barbara steel thi is a fairli odd film in that the present is both in french and english and switch back and forth a few time perhap thi is done becaus bit of dialog were lost it s also rather dark and claustrophob be that one doesn t see much beyond a small circl of light that candl and such gener plu there s a feel of dread and impend doom pretti much at all time thi version on synaps is also uncensor and i wonder what might be censor in a film from 1964 until i saw the topless scene i guess that might be it overal thi is pretti good and in gloomi black and white barbara steel definit make the movi too 8 out of 10 ',
       'good soundtrack great interest scene love need improv movi '],
      dtype=objec

In [21]:
reviews_vet_tf = TfidfVectorizer(stop_words= 'english')
reviews_matrix_tf = reviews_vet_tf.fit_transform(documents)

In [22]:
print(reviews_matrix_tf.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 2) Consultar conjunto de dados

A nossa consulta ao conjunto de dados consistirá em calcular as distâncias entre nosso conjunto de palavras chave, construído por nós para a consulta, e as reviews retiradas do conjunto de dados. Para isso, utilizaremos a **Distância Cosseno**.

In [23]:
distances = metrics.pairwise_distances(reviews_matrix_tf[:-1], Y=reviews_matrix_tf[-1], metric='cosine')

In [24]:
distances.shape

(1000, 1)

Conforme podemos observar, a matriz gerada pelo cálculo da distância tem dimensões 1000 x 1. Isso acontece porque calculamos a distância apenas entre as reviews **reviews_matrix_tf[:-1]** e a nossa consulta **reviews_matrix_tf[-1]**. Assim, como são 1000 reviews e apenas um documento de consulta, obtemos as dimensões mostradas.

Portanto, para determinarmos quais as 10 reviews mais semelhantes com a nossa consulta, podemos utilizar os valores da matriz *distances*, uma vez que é lá onde estão as distâncias entre a nossa consulta e o restante das reviews.

## 3) Mostrar as 10 reviews mais semelhantes

Uma vez que estamos calculando as distâncias, as 10 reviews mais semelhantes serão aquelas que apresentam as 10 menores distâncias. Portanto, extrairemos os 10 menores valores da linha utilizada.

Fazemos o reshape para transformarmos a matriz obtida em um único vetor.

In [25]:
new_dist = distances.reshape((distances.shape[0],))

In [26]:
most_similar = np.argsort(new_dist)[:10]

In [27]:
# dist_consulta = distances[-1]
# dist_consulta

In [28]:
#most_similar = np.argsort(dist_consulta)[1:11]

Portanto, as reviews mais semelhantes à consulta realizada são as reviews presentes nas posições:

In [29]:
most_similar

array([940, 322, 595, 606, 444, 802, 983, 135, 597,  59])

Aqui podemos ver a consulta depois de ter sido normalizada:

In [30]:
documents[-1]

'good soundtrack great interest scene love need improv movi '

Aqui podemos ver as 10 reviews mais similares:

In [31]:
for r in most_similar:
    print("Review {}:\n{}\n".format(r, documents[r]))

Review 940:
i saw thi movi in 1956 and again on cabl a few day ago the movi hasn t improv with age quit the opposit it s a true spaghetti epic the trojan are heroic and likabl the greek are nasti petti and sneaki so what if pari ran off with the king s wife hey love is love 

Review 322:
a fine effort for an australian show which is probabl not surpris see as there seem to be somewhat of a resurg in qualiti aussi drama dare i compar thi show to the brillianc of love my way no but it is reminisc of earli secret life of us the cast is great gibney work her magic in the first two episod i have seen the british cast is strong also especi the callum and lizzi charact but abe forsyth may be the save light not that it need save if thi show is to get anoth season i wasn t a fan of hi perform in the awesom awesom mark time mini seri a few year back but he wa great as hal in alway greener it also good to see brook satchwel again let hope the show keep improv with each episod 

Review 595:
i ve n